In [3]:
import graphlab
import numpy as np
import itertools

A newer version of GraphLab Create (v1.8.3) is available! Your current version is v1.8.2.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [4]:
dataset = graphlab.SFrame.read_csv('train.csv', verbose=False)

[INFO] GraphLab Create v1.8.2 started. Logging: /tmp/graphlab_server_1457113496.log


In [14]:
def guess_age(ini_df):
    
    # Sampling
    df_age_null = ini_df[ini_df['Age'] == None]
    df_age = ini_df.dropna('Age', how="all")
    df_age_train, df_age_test = df_age.random_split(.8 ,seed=1)
    
    # allfeatures
    all_features = ['Survived', 'Pclass', 'Name','Sex', 'SibSp', 'Parch', 'Fare', 'Ticket', 'Embarked']
    
    #build the model
    model_age = graphlab.linear_regression.create(
        df_age_train, 
        target='Age', 
        features=all_features,
        l1_penalty=0, 
        l2_penalty=0.03, 
        verbose=False,
        max_iterations=100
    )
    
    df = df_age_null
    df['Age'] = model_age.predict(df)
    
    df['Age'] = df['Age'].apply(lambda y: float(round(y)))
    
    #print type(df['Age'])
    #print df_age
    #print df_age.head()
    return df_age.append(df) #df_age

In [15]:
graphlab.canvas.set_target('ipynb')
dataset['Survived'].show(view='Categorical')

In [19]:
print dataset.num_rows()
#dataset_2=dataset.dropna('Age',how="all")
#print dataset_2.num_rows()

dataset_age = guess_age(dataset) 

print dataset_age.num_rows()
#print dataset_age.tail()

train_data,test_data = dataset_age.random_split(.8, seed=0)

891
891


In [20]:
all_features=['Pclass','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked']

# Selection

## Permutation

In [21]:
tab_general=[]

#for i in range(2, len(all_features)):
for i in range(2, 2):
    print i
    list_permu_current=list(itertools.permutations(all_features, i))
    accuracy_tab=[]
    
    for x in range(0, len(list_permu_current)):
        array_feature_current=list(list_permu_current[x])
        print array_feature_current
        current_model=graphlab.logistic_classifier.create(train_data,target='Survived',
                                                     features=array_feature_current,
                                                    validation_set=test_data,verbose=False)
        accuracy_current=current_model.evaluate(test_data)['accuracy']
        accuracy_tab.append(accuracy_current)
        print accuracy_current
        print str(x) +'/'+ str(len(list_permu_current))
    
    print max(accuracy_tab)
    index_max=accuracy_tab.index(max(accuracy_tab)) 
    print list_permu_current[index_max]
    tab_general.append(list_permu_current[index_max])

## Backward

In [25]:
def getArrayAccuracy(all_features,target_feature,train_data,test_data,method):
    all_features=np.array(all_features)
    number_feature=len(all_features)
    array_feature_current_gen=all_features

    accuracy_general=[]
    while number_feature>2:
        print '-------------------------------'
        accuracy_tab=[]
        print "array_feature_current_gen : " + str(array_feature_current_gen)

        for x in range(0, len(array_feature_current_gen)):

            array_feature_current = np.delete(array_feature_current_gen, x)
            #print array_feature_current
            if(method=="LOG_CLA"):
                current_model=graphlab.logistic_classifier.create(train_data,target=target_feature,features=array_feature_current,validation_set=test_data,verbose=False)
                accuracy_current=current_model.evaluate(test_data)['f1_score']
                
            accuracy_tab.append(accuracy_current)
            #print accuracy_current
            #print str(x+1) +'/'+ str(len(array_feature_current_gen))

        accuracy_general.append(max(accuracy_tab))
        print "max : "+ str(max(accuracy_tab))
        index_max=accuracy_tab.index(max(accuracy_tab)) 
        #print index_max
        print "element a supprimer : " +array_feature_current_gen[index_max]

        number_feature=number_feature-1
        array_feature_current_gen=np.delete(array_feature_current_gen, index_max)
    
    return accuracy_general

In [26]:
accuracy_general=getArrayAccuracy(all_features,"Survived",train_data,test_data,"LOG_CLA")

-------------------------------
array_feature_current_gen : ['Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin' 'Embarked']
max : 0.776315789474
element a supprimer : Cabin
-------------------------------
array_feature_current_gen : ['Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Embarked']
max : 0.78431372549
element a supprimer : Pclass
-------------------------------
array_feature_current_gen : ['Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Embarked']
max : 0.78431372549
element a supprimer : Age
-------------------------------
array_feature_current_gen : ['Sex' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Embarked']
max : 0.78431372549
element a supprimer : SibSp
-------------------------------
array_feature_current_gen : ['Sex' 'Parch' 'Ticket' 'Fare' 'Embarked']
max : 0.78431372549
element a supprimer : Parch
-------------------------------
array_feature_current_gen : ['Sex' 'Ticket' 'Fare' 'Embarked']
max : 0.78431372549
element a supprimer : Fare
---------------------------

In [ ]:
import matplotlib.pyplot as plt
t = np.arange(len(all_features), 2, -1)
plt.plot(t, accuracy_general)
plt.xlim(len(all_features), 2) 
plt.show()

In [ ]:
accuracy_general.show(view='Categorical')